
## Preliminar settings

Improting the libraries we are using for this Notebook, importing the files. 

In [14]:
import pandas as pd
import numpy as np
import time
import json
import re
import folium 
from geopy import distance
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from stemming.porter2 import stem
from nltk.stem import PorterStemmer
import csv
from collections import Counter
import heapq

import IPython

When we started the homework, we began working with .tsv files, splitting the .csv dataset in one .tsv each record

In [ ]:
df = pd.read_csv('Airbnb_Texas_Rentals.csv')

df.drop('Unnamed: 0', axis=1, inplace=True)

tsvs = np.array_split(df, df.shape[0])
for i, tsv in enumerate(tsvs):
    tsv.to_csv('doc_' + '{}.tsv'.format(i), sep='\t', index=False, header=False, index_label=False)
    
big_set_of_words = set()
with open("Airbnb_Texas_Rentals.csv", encoding="utf-8") as csv_file:
  csv_reader = csv.reader(csv_file, delimiter=',')
  for row in csv_reader:
    word_with_hash = (row[0], hash(row[0]))
    big_set_of_words.add(word_with_hash)

with open("vocabulary.csv", mode = "w", newline='', encoding="utf-8") as output_file:
  csv_writer = csv.writer(output_file, delimiter=',')
  for element in big_set_of_words:
    csv_writer.writerow(element)

But then, while working on the files we've seen that it was consuming way too much time to iterate opening and reading each time over all the documents. We've worked directly on dataframes and dictionaries.

In [2]:
data = pd.read_csv('Airbnb_Texas_Rentals.csv')
data = data.rename(index=str, columns={"Unnamed: 0": "number"})

After importing data we want to create a new column containing list of words from title and description

In [3]:
data['title_description'] = data.title.str.strip().str.split('[\W_]+') + data.description.str.strip().str.split('[\W_]+')

Now we are creating a Dataframe that will contain advertisment number and each word in it row by row

In [4]:
rows = list()
for row in data[['number', 'title_description']].iterrows():
    r = row[1]
    try:
        for word in r.title_description:
            rows.append((r.number, word))
    except:
        rows.append((r.number, r.title_description))
        
words = pd.DataFrame(rows, columns=['post', 'word'])


Now we want to clean the data as much as possible. We will clear empty cells, lowercase each letter, stem all the words, and removing the stopwords. The result will rapresent our **vocabulary** from which we will start constructing the first inverted index.

In [5]:
# clearing empty cells
words = words[words.word.str.len() > 0]
# changing all the letters to lowercase
words['word'] = words.word.str.lower()
# stemming the words
words['word'] = words.word.apply(lambda x: stem(x))
# assigning stopwords list
stop = stopwords.words('english')
# removing stopwords and creating a new dataframe 
filtered = words[words['word'].map(lambda x: x not in stop)]

In [6]:
# creating a dataframe which contains each word and documents in which they appear
# pre step for creating the first inverted index

z = filtered.groupby( [ 'word','post'] ).size().to_frame(name = 'count').reset_index()

In [7]:
# for now removing count (number of times a word appear in a given document)

z_without = z.drop('count', axis=1)

In [8]:
# creating the first inverted index

index = z_without.groupby('word')['post'].apply(lambda g: g.values.tolist()).to_dict()

In [9]:
index

{'0': [313,
  775,
  1214,
  1475,
  1698,
  1836,
  2372,
  2415,
  2852,
  3500,
  3550,
  3858,
  4422,
  4570,
  4706,
  5057,
  5725,
  5973,
  5987,
  5994,
  6039,
  8011,
  8101,
  8176,
  8926,
  9172,
  9244,
  9378,
  9384,
  10062,
  10078,
  10365,
  10763,
  10980,
  11032,
  11781,
  11824,
  11970,
  12243,
  12418,
  14440,
  14560,
  14841,
  14992,
  14994,
  15321,
  15422,
  15516,
  15810,
  15988,
  16167,
  16201,
  16517,
  16591,
  16608,
  17009,
  17110,
  17226,
  17364,
  17561,
  17848,
  17901,
  18073],
 '00': [607,
  841,
  1324,
  1409,
  1456,
  1487,
  1713,
  1762,
  1959,
  2055,
  3247,
  3385,
  3529,
  3537,
  3639,
  3759,
  4105,
  4192,
  4370,
  4384,
  4497,
  4624,
  4809,
  5192,
  5584,
  5906,
  6336,
  6344,
  6422,
  6947,
  7246,
  7279,
  7310,
  7410,
  7799,
  7956,
  7972,
  8216,
  8521,
  8885,
  9062,
  9244,
  9253,
  9436,
  10044,
  10693,
  11380,
  11574,
  11718,
  12060,
  12409,
  12442,
  12524,
  12578,
  12846,
  1

# First Search Engine

***

Taken as an input a query that has to been preprocessed, the search engine will return to the user who's searching the Title, Description, City and URL of the advertisments that contain **ALL** the words that are present in the query.

First of all we need to clean the input query. We defined this function to do so:

In [10]:
def clean(sentence):
    #removing punctuation
    s = re.sub(r'[^\w\s]','',sentence)
    #removing stopwords
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(s)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    #stemming and lowering 
    newS=[]
    for elem in filtered_sentence:
        elem = stem(elem)
        elem = elem.lower()
        newS.append(elem)
    return newS
        

### Query sample

We have used this example, but the input can be even aske to be typed by the user.

In [11]:
query = "place with a big garden"

In [12]:
Search_query = clean(query)
Search_query

['place', 'big', 'garden']

The following code will return the row number of the advertisments which contain al the words inserted in the query.

In [13]:
rows = [index[w] for w in Search_query]
common_rows = rows[0]
for r in rows[1:]:
    common_rows = list(set(common_rows) & set(r))

We want to get just the interested columns of the rows (advertisments) that contain all the words of the search query.

In [14]:
pd.set_option('display.max_colwidth', 120)
df = data.loc[data["number"].isin(common_rows), ["city","description", "title", "url"]]

In [15]:
df.head()

,city,description,title,url
3947,Manchaca,"A stone throw south of the Austin City Limits, this cabin is situated in 2.5 wooded acres. Luxurious and quiet. It i...",BREATHE DEEPLY A Cozy Austin Cabin,https://www.airbnb.com/rooms/728502?location=Colorado%20River%2C%20TX
5454,Austin,"HOLIDAYS, LOCAL EVENTS, HIGH DEMAND WEEKENDS, DATES WITH LOW AVAILABILITY MAY BE SUBJECT TO HIGHER RATES, NIGHT MINI...",2 blocks to Rainey! Walk downtown!,https://www.airbnb.com/rooms/1250575?location=Colorado%20River%2C%20TX
9261,Manchaca,"A stone throw south of the Austin City Limits, this cabin is situated in 2.5 wooded acres. Luxurious and quiet. It i...",BREATHE DEEPLY A Cozy Austin Cabin,https://www.airbnb.com/rooms/728502?location=Buda%2C%20TX
9890,Roanoke,My place is close to Walking distance/across the street from Texas motor speedway!!. You’ll love my place because of...,Welcome home!,https://www.airbnb.com/rooms/15603065?location=Bridgeport%2C%20TX
10402,Kyle,"My place is close to family-friendly activities, hill country, rivers, nightlife. You’ll love my place because of th...",New! A little bit country close to town/The Garden,https://www.airbnb.com/rooms/17270667?location=Buda%2C%20TX


# Creating the second inverted index

In [16]:
# counting occurence of each word in each document

counts = filtered.groupby('post')\
    .word.value_counts()\
    .to_frame()\
    .rename(columns={'word':'n_w'})

In [17]:
# counting number of words in each document

word_sum = counts.groupby(level=0)\
    .sum()\
    .rename(columns={'n_w': 'n_d'})

In [18]:
# calculating term frequency

tf = counts.join(word_sum)
tf['tf'] = tf.n_w/tf.n_d

In [19]:
# counting the number of documents

document_count = words.post.nunique()

In [20]:
# counting the number of occurence of each word in the whole dataset

idf = words.groupby('word')\
    .post\
    .nunique()\
    .to_frame()\
    .rename(columns={'post':'i_d'})\
    .sort_values('i_d')

In [21]:
# calculating idf term

idf['idf'] = np.log(document_count/idf.i_d.values)
tf_idf = tf.join(idf)

# calculating tf_idf 

tf_idf['tf_idf'] = tf_idf.tf * tf_idf.idf

In [22]:
idf = tf_idf.drop(['n_w', 'n_d', 'tf', 'i_d','tf_idf'], axis=1)

idf = idf['idf']
idf = idf.to_frame(name = 'idf').reset_index()
idf.drop(['post', 'word'], axis=1, inplace=True)

hz = filtered.groupby('post').word.value_counts().to_frame(name = 'count').reset_index()
hz.drop('count', axis=1, inplace=True)
idf_frame = hz.join(idf)

In [23]:
tfidf = tf_idf.drop(['n_w', 'n_d', 'tf', 'i_d','idf'], axis=1)

tfidf = tfidf['tf_idf']
tfidf = tfidf.to_frame(name = 'tf_idf').reset_index()
tfidf.drop(['post', 'word'], axis=1, inplace=True)


tfidf_frame = idf_frame.join(tfidf)

In [24]:
pre = filtered.groupby('post').word.value_counts().to_frame(name = 'count').reset_index()
pre.drop('count', axis=1, inplace=True)

In [25]:
# getting tf_idf column from the resultant final dataframe

h = tf_idf.drop(['n_w', 'n_d', 'tf', 'i_d', 'idf'], axis=1)
h = h['tf_idf']
h = h.to_frame(name = 'tf_idf').reset_index()
h.drop(['post', 'word'], axis=1, inplace=True)

In [26]:
# concatenating two dataframes in order to get a dataframe in form of (post 	word 	tf_idf)

y = filtered.groupby('post').word.value_counts().to_frame(name = 'count').reset_index()
y.drop('count', axis=1, inplace=True)
tf_idf_frame = y.join(h)

In [27]:
# creating a dictionary for the second inverted index

inverted_index = tf_idf_frame.set_index('word').apply(tuple, 1)\
             .groupby(level=0).agg(lambda x: list(x.values))\
             .to_dict()

In [28]:
inverted_index

{'0': [(313.0, 0.06669417396100341),
  (775.0, 0.09945622432781209),
  (1214.0, 0.13497630444488784),
  (1475.0, 0.17443091651339354),
  (1698.0, 0.059673734596687254),
  (1836.0, 0.059673734596687254),
  (2372.0, 0.09448341311142149),
  (2415.0, 0.1667354349025085),
  (2852.0, 0.09608482689297101),
  (3500.0, 0.23620853277855372),
  (3550.0, 0.1667354349025085),
  (3858.0, 0.07459216824585907),
  (4422.0, 0.1667354349025085),
  (4570.0, 0.07459216824585907),
  (4706.0, 0.059673734596687254),
  (5057.0, 0.09143556107556919),
  (5725.0, 0.1453590970944946),
  (5973.0, 0.14172511966713225),
  (5987.0, 0.1667354349025085),
  (5994.0, 0.09608482689297101),
  (6039.0, 0.06669417396100341),
  (8011.0, 0.1667354349025085),
  (8101.0, 0.13183732062058812),
  (8176.0, 0.1667354349025085),
  (8926.0, 0.2180386456417419),
  (9172.0, 0.1453590970944946),
  (9244.0, 0.07660817279304445),
  (9378.0, 0.17443091651339354),
  (9384.0, 0.09143556107556919),
  (10062.0, 0.08857819979195765),
  (10078.0, 

In [29]:
tfidf_frame.head()

,post,word,idf,tf_idf
0,1,privat,1.278876,0.134619
1,1,10,2.224322,0.117070
2,1,addit,3.615695,0.190300
3,1,airport,2.219269,0.116804
4,1,avail,2.412130,0.126954


# Second Search Engine

As in the first search engine, we need to preprocess the input query.

### Query sample

In [30]:
query2 = 'a beautiful house with garden'

In [31]:
Search_query2 = clean(query)
query_tf = Counter(Search_query2)
query_tf

Counter({'place': 1, 'big': 1, 'garden': 1})

In [32]:
query_tf_lst = []
for k,v in query_tf.items():
    query_tf[k] = v/len(query)
    query_tf_lst.append(v/len(query))

In [33]:
query_tf

Counter({'place': 0.043478260869565216,
         'big': 0.043478260869565216,
         'garden': 0.043478260869565216})

In [34]:
query_idf = {}
query_idf_lst = []


for k,v in query_tf.items():
    try:
        query_idf[k] = tfidf_frame[tfidf_frame['word']==k]['idf'].iloc[0]
        query_idf_lst.append(tfidf_frame[tfidf_frame['word']==k]['idf'].iloc[0])
    except:
        query_idf[k] = 0
        query_idf_lst.append(0)

In [35]:
query_idf

{'place': 1.4376626238621804,
 'big': 2.7848249990370464,
 'garden': 3.413544578541615}

In [36]:
# obtaining tf_idf of words in query

query_tfidf = np.array(query_tf_lst)*np.array(query_idf_lst)

In [37]:
from heapq import heappush, heappop

In [38]:
# calculating cosine similarity

heap = []

for i in range(1,3):
    xxx = tfidf_frame[tfidf_frame['post']==i]
    tf_lst = []
    for k,v in query_tf.items():

        try:
            tf_lst.append(xxx[xxx['word']==k]['tf_idf'].iloc[0])
        except:
            tf_lst.append(0)
    similarity =  np.dot(query_tfidf,np.array(tf_lst))/(np.sum(query_tfidf**2)*np.sum(np.array(tf_lst)**2))
    if not np.isnan(similarity):
        heappush(heap, (similarity,i))

C:\Users\Marco\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  


In [45]:
# most similar 5 files 
# format: (similarity, document id)

most_similar = heapq.nlargest(5,heap)
most_similar

[(70.68878500401094, 2)]

In [40]:
similar_rows = []
for i in most_similar:
    similar_rows.append(i[1])

In [41]:
data.loc[data["number"].isin(similar_rows), ["city","description", "title", "url"]]

,city,description,title,url
1,San Antonio,"Stylish, fully remodeled home in upscale NW – Alamo Heights Area. \n\nAmazing location - House conveniently located ...",Unique Location! Alamo Heights - Designer Inspired,https://www.airbnb.com/rooms/17481455?location=Cibolo%2C%20TX


# Visualization

In order to provide a better solution, we imagine the user could do a research by location which is the feature that the most part of B&B searching engines have today. We want the user to specify latitude and longitude and a radius inside of which finding different solutions

In [42]:
data = pd.read_csv('Airbnb_Texas_Rentals.csv', header = 0, sep =",")
data = data.dropna().drop_duplicates(subset=["title", "description", "city"])
data.reset_index(drop=True, inplace=True)
data

,Unnamed: 0,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url
0,1,$27,2,Humble,May 2016,Welcome to stay in private room with queen bed and detached private bathroom on the second floor. Another private be...,30.020138,-95.293996,2 Private rooms/bathroom 10min from IAH airport,https://www.airbnb.com/rooms/18520444?location=Cleveland%2C%20TX
1,2,$149,4,San Antonio,November 2010,"Stylish, fully remodeled home in upscale NW – Alamo Heights Area. \n\nAmazing location - House conveniently located ...",29.503068,-98.447688,Unique Location! Alamo Heights - Designer Inspired,https://www.airbnb.com/rooms/17481455?location=Cibolo%2C%20TX
2,3,$59,1,Houston,January 2017,'River house on island close to the city' \nA well maintained river house off the San Jacinto river with extra room ...,29.829352,-95.081549,River house near the city,https://www.airbnb.com/rooms/16926307?location=Beach%20City%2C%20TX
3,4,$60,1,Bryan,February 2016,Private bedroom in a cute little home situated in the coveted Garden Acres neighborhood in Bryan. The bedroom has it...,30.637304,-96.337846,Private Room Close to Campus,https://www.airbnb.com/rooms/11839729?location=College%20Station%2C%20TX
4,5,$75,2,Fort Worth,February 2017,Welcome to our original 1920's home. We recently purchased this 1922 home just a mile outside of downtown Fort Worth...,32.747097,-97.286434,The Porch,https://www.airbnb.com/rooms/17325114?location=Colleyville%2C%20TX
5,6,$250,4,Conroe,August 2016,"My place is close to Lake Conroe, family-friendly activities, nightlife. You’ll love my place because of the outdoor...",30.370455,-95.385319,Gorgeous Home with Country Setting,https://www.airbnb.com/rooms/14466133?location=Cleveland%2C%20TX
6,7,$129,3,Cedar Creek,March 2016,Rustic country retreat on 8 acres southeast of Austin. Converted modular home with all the amenities needed while of...,30.109838,-97.473417,Cozy and Quaint Country Retreat with Acreage.,https://www.airbnb.com/rooms/12491762?location=Cedar%20Creek%2C%20TX
7,8,$25,1,Fort Worth,January 2016,This is a beautiful bedroom with a queen size bed and closet. We do not have pets and the house is always clean. The...,32.689611,-97.298161,Friendly Private Room in َQuiet Neighborhood,https://www.airbnb.com/rooms/18977363?location=Cleburne%2C%20TX
8,9,$345,3,Rockport,February 2016,First class and comfortable condo with the bests views from bottom deck and master bedroom patio.\r\nPier is just a ...,28.103726,-97.025832,608 - Bayfront Condos,https://www.airbnb.com/rooms/17559848?location=Bayside%2C%20TX
9,10,$72,Studio,San Antonio,August 2013,Private entrance to your own \,29.480646,-98.495692,Cozy Historic Private Studio,https://www.airbnb.com/rooms/1588127?location=Colorado%20River%2C%20TX


In [43]:
print("key in latitude")
lati = input()
if lati != "" :
    lat = float(lati) 
else:
    print("invalid input")

print("keep in longitude")
longi = input()
if longi != "" :
    long = float(longi)
else:
    print("invalid input")

print("input radius:")
radius = input()
if radius != "" :
    radius = float(radius)
else:
    print("invalid input")

key in latitude
29.4722 
keep in longitude
-98.5247
input radius:
15.0


In [44]:
#finding the distance between coordinates
def max_distance(x):
    latitude = x["latitude"]
    longitude = x["longitude"]
    return distance.distance((lat, long), (latitude, longitude)).miles * 1.60934

data["distance"] = data.loc[:, ["latitude", "longitude"]].apply(max_distance, axis=1)
#choose only points within selected radius
data_new = data[data["distance"] <= radius]


#plotting with user's input 
mapp = folium.Map(location=[lat, long], zoom_start = 12)
folium.Marker(location=(lat, long), popup="Center",icon=folium.Icon(color="green")).add_to(mapp)
folium.Circle(location=[lat, long], radius=radius*1000, color='blue', fill=True, fill_color="blue").add_to(mapp)


#plotting the points within the radius 
for i, doc in data_new.iterrows():
    doc_ = data_new[['title', 'description']]
    doc_.columns = ["Title", "Description"]
    html = doc_.to_html(classes='table', index=False, justify="center")
    popup = folium.Popup(html)
    folium.Marker( location=(doc["latitude"], doc["longitude"]), popup=popup).add_to(mapp)


# Save map
mapp.save('mapp.html')

The map shows a visualization of a selected coordinates of the user and plots avaliable houses that are within inputed radius The user's selected corrodinates is coded in green to distinguish it from the rest of the availabe houses.

![alt text](MAP.png)